In [1]:
import fastf1 as ff1
ff1.Cache.enable_cache('cache')  
monza_quali = ff1.get_session(2019, 'Monza', 'Q')
vettel = monza_quali.get_driver('VET')
print(f"Pronto {vettel.name}?")

NotADirectoryError: Cache directory does not exist! Please check for typos or create it first.